In [1]:
import cv2
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root, image_path):
        self.root = root
        self.root.title("Image Cropper")

        # Open the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        max_size = (800, 600)  # Adjust the maximum size as needed
        resized_image = cv2.resize(original_image, max_size)

        # Convert the OpenCV image to a PIL Image
        self.image = Image.fromarray(resized_image)

        self.canvas = tk.Canvas(root, width=self.image.width, height=self.image.height)
        self.canvas.pack()

        self.tk_image = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        self.polygon_points = []
        self.polygon_id = None

        self.canvas.bind("<Button-1>", self.on_click)
        self.root.bind("<Return>", lambda event: self.crop_image())

    def on_click(self, event):
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)

        # Add the clicked point to the list
        self.polygon_points.append((x, y))

        # Draw a circle at the clicked point
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="red")

        # Draw lines between consecutive points to visualize the polygon
        if len(self.polygon_points) > 1:
            self.canvas.delete("polygon_line")
            for i in range(len(self.polygon_points) - 1):
                self.canvas.create_line(
                    self.polygon_points[i],
                    self.polygon_points[i + 1],
                    fill="red",
                    width=2,
                    tags="polygon_line",
                )

    def crop_image(self):
        # Draw a line connecting the last and first points to close the polygon
        if len(self.polygon_points) > 2:
            self.canvas.create_line(
                self.polygon_points[-1],
                self.polygon_points[0],
                fill="red",
                width=2,
                tags="polygon_line",
            )

            # Find the bounding box of the polygon
            xs, ys = zip(*self.polygon_points)
            bbox = (min(xs), min(ys), max(xs), max(ys))

            # Crop the image based on the bounding box
            cropped = self.image.crop(bbox)

            # Show the cropped image in a separate viewer window
            cropped.show()

            # Save the cropped image
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
            if save_path:
                cropped.save(save_path)

if __name__ == "__main__":
    root = tk.Tk()

    # Ask the user to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])

    if file_path:
        cropper = ImageCropper(root, file_path)
        crop_button = tk.Button(root, text="Crop Image", command=cropper.crop_image)
        crop_button.pack()

        # Set the window size to match the image dimensions
        root.geometry(f"{cropper.image.width}x{cropper.image.height}")

        root.mainloop()


# Croping and saving the image in the local

In [4]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root, image_path):
        self.root = root
        self.root.title("Image Cropper")

        # Open the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        max_size = (800, 600)  # Adjust the maximum size as needed
        resized_image = cv2.resize(original_image, max_size)

        # Convert the OpenCV image to a PIL Image
        self.image = Image.fromarray(resized_image)

        self.canvas = tk.Canvas(root, width=self.image.width, height=self.image.height)
        self.canvas.pack()

        self.tk_image = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        self.polygon_points = []
        self.polygon_id = None

        self.canvas.bind("<Button-1>", self.on_click)
        self.root.bind("<Return>", lambda event: self.crop_image())
        self.root.bind("<Control-s>", lambda event: self.save_coordinates())

    def on_click(self, event):
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)

        # Add the clicked point to the list
        self.polygon_points.append((x, y))

        # Draw a circle at the clicked point
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="red")

        # Draw lines between consecutive points to visualize the polygon
        if len(self.polygon_points) > 1:
            self.canvas.delete("polygon_line")
            for i in range(len(self.polygon_points) - 1):
                self.canvas.create_line(
                    self.polygon_points[i],
                    self.polygon_points[i + 1],
                    fill="red",
                    width=2,
                    tags="polygon_line",
                )

    def crop_image(self):
        # Draw a line connecting the last and first points to close the polygon
        if len(self.polygon_points) > 2:
            self.canvas.create_line(
                self.polygon_points[-1],
                self.polygon_points[0],
                fill="red",
                width=2,
                tags="polygon_line",
            )

            # Find the bounding box of the polygon
            xs, ys = zip(*self.polygon_points)
            bbox = (min(xs), min(ys), max(xs), max(ys))

            # Create a mask using the polygon
            mask = np.zeros_like(np.array(self.image), dtype=np.uint8)
            roi_corners = np.array([self.polygon_points], dtype=np.int32)
            cv2.fillPoly(mask, roi_corners, (255, 255, 255))

            # Convert the OpenCV image to a numpy array
            image_np = np.array(self.image)

            # Apply the mask to the original image
            masked_image = cv2.bitwise_and(image_np, mask)

            # Convert the masked image to a PIL Image
            cropped = Image.fromarray(masked_image)

            # Save the cropped image
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
            if save_path:
                cropped.save(save_path)

    def save_coordinates(self):
        # Save the coordinates to a text file
        coordinates_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if coordinates_path:
            with open(coordinates_path, 'w') as file:
                for point in self.polygon_points:
                    file.write(f"{point[0]},{point[1]}\n")

if __name__ == "__main__":
    root = tk.Tk()

    # Ask the user to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])

    if file_path:
        cropper = ImageCropper(root, file_path)
        crop_button = tk.Button(root, text="Crop Image", command=cropper.crop_image)
        crop_button.pack()

        # Set the window size to match the image dimensions
        root.geometry(f"{cropper.image.width}x{cropper.image.height}")

        root.mainloop()


# Following code are based on the croping the image based on the co-ordinate text file

In [5]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root, image_path):
        self.root = root
        self.root.title("Image Cropper")

        # Open the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        max_size = (800, 600)  # Adjust the maximum size as needed
        resized_image = cv2.resize(original_image, max_size)

        # Convert the OpenCV image to a PIL Image
        self.image = Image.fromarray(resized_image)

        self.canvas = tk.Canvas(root, width=self.image.width, height=self.image.height)
        self.canvas.pack()

        self.tk_image = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        self.polygon_points = []
        self.polygon_id = None

        self.canvas.bind("<Button-1>", self.on_click)
        self.root.bind("<Return>", lambda event: self.crop_image())
        self.root.bind("<Control-s>", lambda event: self.save_coordinates())

    def on_click(self, event):
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)

        # Add the clicked point to the list
        self.polygon_points.append((x, y))

        # Draw a circle at the clicked point
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="red")

        # Draw lines between consecutive points to visualize the polygon
        if len(self.polygon_points) > 1:
            self.canvas.delete("polygon_line")
            for i in range(len(self.polygon_points) - 1):
                self.canvas.create_line(
                    self.polygon_points[i],
                    self.polygon_points[i + 1],
                    fill="red",
                    width=2,
                    tags="polygon_line",
                )

    def crop_image(self):
        # Draw a line connecting the last and first points to close the polygon
        if len(self.polygon_points) > 2:
            self.canvas.create_line(
                self.polygon_points[-1],
                self.polygon_points[0],
                fill="red",
                width=2,
                tags="polygon_line",
            )

            # Find the bounding box of the polygon
            xs, ys = zip(*self.polygon_points)
            bbox = (min(xs), min(ys), max(xs), max(ys))

            # Create a mask using the polygon
            mask = np.zeros_like(np.array(self.image), dtype=np.uint8)
            roi_corners = np.array([self.polygon_points], dtype=np.int32)
            cv2.fillPoly(mask, roi_corners, (255, 255, 255))

            # Convert the OpenCV image to a numpy array
            image_np = np.array(self.image)

            # Apply the mask to the original image
            masked_image = cv2.bitwise_and(image_np, mask)

            # Convert the masked image to a PIL Image
            cropped = Image.fromarray(masked_image)

            # Save the cropped image
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
            if save_path:
                cropped.save(save_path)

    def save_coordinates(self):
        # Save the coordinates to a text file
        coordinates_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if coordinates_path:
            with open(coordinates_path, 'w') as file:
                for point in self.polygon_points:
                    file.write(f"{point[0]},{point[1]}\n")

    def load_coordinates_from_file(self, file_path):
        # Load coordinates from a text file
        with open(file_path, 'r') as file:
            lines = file.readlines()
            self.polygon_points = [(float(line.split(',')[0]), float(line.split(',')[1])) for line in lines]

        # Draw the loaded polygon on the canvas
        self.canvas.delete("polygon_line")
        for i in range(len(self.polygon_points) - 1):
            self.canvas.create_line(
                self.polygon_points[i],
                self.polygon_points[i + 1],
                fill="red",
                width=2,
                tags="polygon_line",
            )
        self.canvas.create_line(
            self.polygon_points[-1],
            self.polygon_points[0],
            fill="red",
            width=2,
            tags="polygon_line",
        )

if __name__ == "__main__":
    root = tk.Tk()

    # Ask the user to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])

    if file_path:
        cropper = ImageCropper(root, file_path)
        crop_button = tk.Button(root, text="Crop Image", command=cropper.crop_image)
        crop_button.pack()

        # Ask the user to select a coordinates file
        coordinates_file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])

        if coordinates_file_path:
            # Load coordinates from the file
            cropper.load_coordinates_from_file(coordinates_file_path)

        # Set the window size to match the image dimensions
        root.geometry(f"{cropper.image.width}x{cropper.image.height}")

        root.mainloop()


## Cropping image methodology using the reset button 

In [8]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root, image_path):
        self.root = root
        self.root.title("Image Cropper")

        # Initialize attributes
        self.image_path = image_path
        self.polygon_points = []
        self.polygon_id = None

        # Load image
        self.load_image()

        # Create canvas
        self.canvas = tk.Canvas(root, width=self.image.width, height=self.image.height)
        self.canvas.pack()

        # Display image on canvas
        self.tk_image = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        # Event bindings
        self.canvas.bind("<Button-1>", self.on_click)
        self.root.bind("<Return>", lambda event: self.crop_image())
        self.root.bind("<Control-s>", lambda event: self.save_coordinates())

        # Create buttons
        crop_button = tk.Button(root, text="Crop Image", command=self.crop_image)
        crop_button.pack()

        reset_button = tk.Button(root, text="Reset", command=self.reset_coordinates)
        reset_button.pack()

    def load_image(self):
        # Open the image using OpenCV
        original_image = cv2.imread(self.image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        max_size = (800, 600)  # Adjust the maximum size as needed
        resized_image = cv2.resize(original_image, max_size)

        # Convert the OpenCV image to a PIL Image
        self.image = Image.fromarray(resized_image)

    def on_click(self, event):
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)

        # Add the clicked point to the list
        self.polygon_points.append((x, y))

        # Draw a circle at the clicked point
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="red")

        # Draw lines between consecutive points to visualize the polygon
        if len(self.polygon_points) > 1:
            self.canvas.delete("polygon_line")
            for i in range(len(self.polygon_points) - 1):
                self.canvas.create_line(
                    self.polygon_points[i],
                    self.polygon_points[i + 1],
                    fill="red",
                    width=2,
                    tags="polygon_line",
                )

    def crop_image(self):
        # Draw a line connecting the last and first points to close the polygon
        if len(self.polygon_points) > 2:
            self.canvas.create_line(
                self.polygon_points[-1],
                self.polygon_points[0],
                fill="red",
                width=2,
                tags="polygon_line",
            )

            # Find the bounding box of the polygon
            xs, ys = zip(*self.polygon_points)
            bbox = (min(xs), min(ys), max(xs), max(ys))

            # Create a mask using the polygon
            mask = np.zeros_like(np.array(self.image), dtype=np.uint8)
            roi_corners = np.array([self.polygon_points], dtype=np.int32)
            cv2.fillPoly(mask, roi_corners, (255, 255, 255))

            # Convert the OpenCV image to a numpy array
            image_np = np.array(self.image)

            # Apply the mask to the original image
            masked_image = cv2.bitwise_and(image_np, mask)

            # Convert the masked image to a PIL Image
            cropped = Image.fromarray(masked_image)

            # Save the cropped image
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
            if save_path:
                cropped.save(save_path)

    def save_coordinates(self):
        # Save the coordinates to a text file
        coordinates_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if coordinates_path:
            with open(coordinates_path, 'w') as file:
                for point in self.polygon_points:
                    file.write(f"{point[0]},{point[1]}\n")

    def reset_coordinates(self):
        # Reset the coordinates and clear the canvas
        self.polygon_points = []
        self.canvas.delete("polygon_line")
        self.load_image()  # Load the image again
        self.tk_image = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

if __name__ == "__main__":
    root = tk.Tk()

    # Ask the user to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])

    if file_path:
        cropper = ImageCropper(root, file_path)

        # Set the window size to match the image dimensions
        root.geometry(f"{cropper.image.width}x{cropper.image.height}")

        root.mainloop()
        
